In [ ]:
import pandas as pd                    # Importa a biblioteca pandas para manipulação de dados
import networkx as nx                  # Importa a biblioteca networkx para criação e manipulação de grafos
import matplotlib.pyplot as plt        # Importa a biblioteca matplotlib para plotagem dos gráficos

# Leitura do dataset a partir do arquivo CSV
# 'credit_card_fraud_detection.csv' é o nome do arquivo
# sep=',' especifica que o separador de colunas no CSV é a vírgula
df = pd.read_csv('credit_card_fraud_detection.csv', sep=',')

# Filtrar apenas as transações fraudulentas e dos países desejados (USA ou Brazil)
# Cada condição é avaliada element-wise e combinadas com & (e) e | (ou)
df = df[(df['Fraudulent'] == 'Yes') & ((df['Country'] == 'USA') | (df['Country'] == 'Brazil'))]

# Verifica se as colunas necessárias existem no DataFrame
colunas_necessarias = ['User ID', 'Country', 'Merchant Category', 'Fraudulent', 'Transaction Amount']
if not all(col in df.columns for col in colunas_necessarias):
    raise ValueError(f"O arquivo precisa conter as colunas: {colunas_necessarias}")

# Agrupa os dados por "Country" e "Merchant Category"
# e soma os valores de "Transaction Amount" para cada grupo.
# Dessa forma, cada categoria de mercado aparecerá apenas uma vez para cada país.
grouped = df.groupby(['Country', 'Merchant Category'])['Transaction Amount'].sum().reset_index()

# Criação dos grafos separados por país usando os dados agregados
country_graphs = {}

# Itera sobre cada país único no DataFrame agrupado
for country in grouped['Country'].unique():
    G = nx.Graph()  # Cria um novo grafo não direcionado para o país atual
    # Filtra o DataFrame agrupado para obter os dados do país atual
    df_country = grouped[grouped['Country'] == country]
    
    # Itera sobre cada linha do DataFrame filtrado
    for _, row in df_country.iterrows():
        merchant = row['Merchant Category']         # Obtém a categoria de mercado
        total_amount = row['Transaction Amount']      # Obtém a soma dos valores (peso da aresta)
        
        # Adiciona o nó representando a categoria de mercado com atributo 'tipo' como 'merchant'
        G.add_node(merchant, tipo='merchant')
        # Adiciona o nó representando o país com atributo 'tipo' como 'pais'
        G.add_node(country, tipo='pais')
        # Cria uma aresta entre a categoria de mercado e o país, atribuindo o valor agregado como peso
        G.add_edge(merchant, country, weight=total_amount)
    
    # Armazena o grafo criado no dicionário usando o nome do país como chave
    country_graphs[country] = G

# Função para plotar os grafos com espessura das arestas proporcional ao peso (soma dos valores)
def plot_graph(G, country):
    # Calcula as posições dos nós utilizando o layout "spring" (layout de força dirigida)
    # O parâmetro seed=42 garante reprodutibilidade na disposição dos nós
    pos = nx.spring_layout(G, seed=42)
    
    # Seleciona os nós do tipo 'merchant' (categorias de mercado)
    merchants = [node for node, attr in G.nodes(data=True) if attr['tipo'] == 'merchant']

    # Desenha os nós das categorias de mercado:
    # pos: posições dos nós calculadas anteriormente
    # nodelist: lista dos nós a serem desenhados (nós do tipo merchant)
    # node_color: cor dos nós (azul)
    # node_size: tamanho dos nós (100)
    nx.draw_networkx_nodes(G, pos, nodelist=merchants, node_color='blue', node_size=100)
    
    # Desenha o nó do país:
    # nodelist: lista contendo apenas o nó do país
    # node_color: cor do nó (vermelho)
    # node_size: tamanho do nó (200)
    nx.draw_networkx_nodes(G, pos, nodelist=[country], node_color='red', node_size=200)
    
    # Obtém os pesos das arestas armazenados no atributo 'weight'
    weights = nx.get_edge_attributes(G, 'weight')
    # Converte os valores dos pesos para uma lista para normalização
    all_weights = list(weights.values())
    
    # Determina o peso mínimo e máximo para normalizar os valores
    min_weight = min(all_weights)
    max_weight = max(all_weights)
    
    # Normaliza os pesos para definir a espessura das arestas
    if max_weight == min_weight:
        widths = [1 for _ in all_weights]
    else:
        # A fórmula mapeia os pesos para uma faixa (por exemplo, de 1 a 5)
        widths = [(weight - min_weight) / (max_weight - min_weight) * 4 + 1 for weight in all_weights]
    
    # Desenha as arestas:
    # edgelist: lista de arestas a serem desenhadas (as chaves do dicionário weights)
    # width: lista de larguras para cada aresta, baseada na normalização dos pesos
    # edge_color: cor das arestas (cinza)
    # alpha: transparência das arestas (0.7)
    nx.draw_networkx_edges(G, pos, edgelist=weights.keys(), width=widths, edge_color='gray', alpha=0.7)
    
    # Desenha os rótulos (nomes) dos nós com fonte tamanho 8
    nx.draw_networkx_labels(G, pos, font_size=8)
    
    # Define o título do gráfico, indicando o país
    plt.title(f"Grafo - {country}")
    # Desativa os eixos para uma visualização mais limpa
    plt.axis('off')
    # Exibe o gráfico
    plt.show()

# Exibe informações dos grafos
print("Grau dos vértices dos países - Transações Fraudulentas:")
for country, G in country_graphs.items():
    grau_pais = G.degree(country)   # Calcula o grau do nó do país
    print(f"{country}: {grau_pais}")

print("\nSoma dos pesos das arestas incidentes em cada país:")
for country, G in country_graphs.items():
    # Obtém as arestas incidentes no nó do país, com seus dados
    incident_edges = G.edges(country, data=True)
    # Soma os pesos ('weight') de cada aresta incidente
    total_weight = sum(data['weight'] for _, _, data in incident_edges)
    print(f"{country}: {total_weight}")

# Plota os grafos para cada país
for country, G in country_graphs.items():
    plot_graph(G, country)


In [ ]:
# Inicializando dataset e visualizando itens
df = pd.read_csv('credit_card_fraud_detection.csv')
df.head()

In [101]:
# Criando função auxiliar para desenhar grafos
def criar_grafo(df, pais):
    grafo = nx.Graph()
    grafo.add_node(pais)

    # Inserindo vértices e arestas
    for i in range(len(df)):
        if df.iloc[i]['Country'] == pais:
            user_id = df.iloc[i]['User ID']
            transaction_amount = df.iloc[i]['Transaction Amount']
            grafo.add_node(user_id)
            grafo.add_edge(user_id, pais, weight=transaction_amount)

    # Retornando grafo
    return grafo

In [106]:
# Filtrando por dados fraudulentos e selecionando países
df_fraudulento = df[df['Fraudulent'] == 'Yes']
paises = df_fraudulento['Country'].unique()

In [ ]:
# Visualizando grafos
for pais in paises:
    grafo = criar_grafo(df_fraudulento, pais)
    
    # Dimensionando figura e ajustando cores
    plt.figure(figsize = (8, 4))
    cor = ['orange' if node == pais else 'teal' for node in grafo.nodes()]

    # Dando nome e visualizando grafo
    nx.draw(grafo, with_labels = True, node_color = cor, edge_color = "gray", font_size = 10, node_size = 1000)
    plt.title(f"Fraudulent graphs in {pais}")
    plt.show()

In [ ]:
# Armazenando grafos
grafos = []
for pais in paises:
    grafos += [criar_grafo(df_fraudulento, pais)]
    print(criar_grafo(df_fraudulento, pais))


In [ ]:
# Visualizando quantidade de fraudes em cada país com base no grau
for grafo in grafos:
    print(grafo.degree(list(grafo.nodes())[0]))